# Eat Safe, Love

## Notebook Set Up

In [14]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [15]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our database
collections = db.list_collection_names()
pprint(collections)

['establishments']


In [18]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [27]:
# Find establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
result = establishments.find(query)

# Use count_documents to display the number of documents in the result
num_documents = establishments.count_documents(query)
print(f"Number of documents with a hygiene score of 20: {num_documents}")

# Display the first document in the results using pprint
pprint(result)

Number of documents with a hygiene score of 20: 82


In [28]:
# Convert the result to a Pandas DataFrame
import pandas as pd
df = pd.DataFrame(result)

# Print the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 82
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  658e2463c5d296723ef00648   110681                  0   
1  658e2463c5d296723ef009c9   612039                  0   
2  658e2463c5d296723ef00cd4   730933                  0   
3  658e2463c5d296723ef00ec1   172735                  0   
4  658e2463c5d296723ef00ed0   172953                  0   
5  658e2463c5d296723ef0186b   512854                  0   
6  658e2463c5d296723ef01a90  1537089                  0   
7  658e2463c5d296723ef02fba   155648                  0   
8  658e2464c5d296723ef03401  1012883                  0   
9  658e2464c5d296723ef03c11   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [37]:
# Find establishments with London as the Local Authority and RatingValue >= 4
querys = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}
results = establishments.find(query)

# Use count_documents to display the number of documents
num_documents = establishments.count_documents(query)
print(f"Number of documents in London with RatingValue >= 4: {num_documents}")

# Display the first document in the results using pprint
pprint(results)


Number of documents in London with RatingValue >= 4: 82


In [38]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 82
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  658e2463c5d296723ef00648   110681                  0   
1  658e2463c5d296723ef009c9   612039                  0   
2  658e2463c5d296723ef00cd4   730933                  0   
3  658e2463c5d296723ef00ec1   172735                  0   
4  658e2463c5d296723ef00ed0   172953                  0   
5  658e2463c5d296723ef0186b   512854                  0   
6  658e2463c5d296723ef01a90  1537089                  0   
7  658e2463c5d296723ef02fba   155648                  0   
8  658e2464c5d296723ef03401  1012883                  0   
9  658e2464c5d296723ef03c11   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [45]:
# Specify the latitude and longitude
latitude = 51.083812 
longitude = 1.195625 

# Define the degree search
degree_search = 0.01

# Construct the query
query1 = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

# Specify the sort order by hygiene score
sort = [("scores.Hygiene", 1)]
# Perform the query and sort
result1 = establishments.find(query).sort(sort)

# Print the results
pprint(result1)

In [47]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(result1)
print(f"Number of rows in the DataFrame: {len(df)}")
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 82
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  658e2463c5d296723ef00648   110681                  0   
1  658e2463c5d296723ef009c9   612039                  0   
2  658e2463c5d296723ef00cd4   730933                  0   
3  658e2463c5d296723ef00ec1   172735                  0   
4  658e2463c5d296723ef00ed0   172953                  0   
5  658e2463c5d296723ef0186b   512854                  0   
6  658e2463c5d296723ef01a90  1537089                  0   
7  658e2463c5d296723ef02fba   155648                  0   
8  658e2464c5d296723ef03401  1012883                  0   
9  658e2464c5d296723ef03c11   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [58]:
# Define the pipeline stages
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Execute the pipeline
result2 = establishments.aggregate(pipeline)

# Print the number of documents in the result
count_documents = sum(1 for _ in result2)
print(f"Number of documents in the result: {count_documents}")

# Re-run the pipeline to reset the cursor
result3 = establishments.aggregate(pipeline)

# Print the first 10 results
print("First 10 results:")
for i, document in enumerate(result3):
    pprint(document)
    if i == 9:  # Stop after printing the first 10 results
        break

Number of documents in the result: 56
First 10 results:
{'_id': 'Thanet', 'count': 2260}
{'_id': 'Greenwich', 'count': 1764}
{'_id': 'Maidstone', 'count': 1426}
{'_id': 'Newham', 'count': 1422}
{'_id': 'Swale', 'count': 1372}
{'_id': 'Chelmsford', 'count': 1360}
{'_id': 'Medway', 'count': 1344}
{'_id': 'Bexley', 'count': 1214}
{'_id': 'Southend-On-Sea', 'count': 1172}
{'_id': 'Tendring', 'count': 1084}


In [59]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result3)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 46
                    _id  count
0            Colchester    996
1       Tunbridge Wells    982
2  Folkestone and Hythe    960
3            Eastbourne    956
4              Hastings    928
5               Bromley    920
6               Ashford    854
7              Havering    794
8              Dartford    766
9             Braintree    764
